Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [10]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from math import sqrt

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 784) (100000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic Regression

In [11]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # In this cell, add the l2 norm of the weights to the loss
  beta = 0.01
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (
      beta * (tf.nn.l2_loss(weights)))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [12]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 46.337749
Training accuracy: 10.0%
Validation accuracy: 13.9%
Loss at step 100: 11.790774
Training accuracy: 74.1%
Validation accuracy: 73.6%
Loss at step 200: 4.466267
Training accuracy: 79.3%
Validation accuracy: 77.6%
Loss at step 300: 1.971178
Training accuracy: 82.1%
Validation accuracy: 80.3%
Loss at step 400: 1.123308
Training accuracy: 83.7%
Validation accuracy: 81.5%
Loss at step 500: 0.832373
Training accuracy: 84.3%
Validation accuracy: 82.2%
Loss at step 600: 0.731369
Training accuracy: 84.5%
Validation accuracy: 82.3%
Loss at step 700: 0.695969
Training accuracy: 84.6%
Validation accuracy: 82.5%
Loss at step 800: 0.683467
Training accuracy: 84.6%
Validation accuracy: 82.5%
Test accuracy: 89.0%


### 2-Layer

In [19]:
batch_size = 128
hidden_layer_nodes = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relus = tf.nn.relu(layer1)
  logits = tf.matmul(relus, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (
      beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_valid_dataset, weights1) + biases1
          ), weights2)
      + biases2)


  test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_test_dataset, weights1) + biases1
          ), weights2)
      + biases2)

In [20]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 744.613159
Minibatch accuracy: 11.7%
Validation accuracy: 34.3%
Minibatch loss at step 500: 190.447495
Minibatch accuracy: 89.8%
Validation accuracy: 81.8%
Minibatch loss at step 1000: 114.303352
Minibatch accuracy: 96.9%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 69.741089
Minibatch accuracy: 98.4%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 42.093609
Minibatch accuracy: 99.2%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 25.554817
Minibatch accuracy: 100.0%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 15.586330
Minibatch accuracy: 99.2%
Validation accuracy: 85.3%
Minibatch loss at step 3500: 9.558954
Minibatch accuracy: 100.0%
Validation accuracy: 85.9%
Minibatch loss at step 4000: 5.950611
Minibatch accuracy: 98.4%
Validation accuracy: 86.0%
Minibatch loss at step 4500: 3.725799
Minibatch accuracy: 99.2%
Validation accuracy: 86.4%
Minibatch loss at step 5000: 2.418855
Minibatch accuracy: 96.9%
Valida

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [41]:
batch_size = 32
hidden_layer_nodes = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relus = tf.nn.relu(layer1)
  logits = tf.matmul(relus, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (
      beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_valid_dataset, weights1) + biases1
          ), weights2)
      + biases2)


  test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_test_dataset, weights1) + biases1
          ), weights2)
      + biases2)

In [42]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    
    # BAD, only allow offsets between 0 and 4, lead to overfitting
    offset = ((step * batch_size) % (train_labels.shape[0] - batch_size)) % 5
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 699.341797
Minibatch accuracy: 9.4%
Validation accuracy: 20.5%
Minibatch loss at step 500: 191.928391
Minibatch accuracy: 100.0%
Validation accuracy: 38.4%
Minibatch loss at step 1000: 116.396263
Minibatch accuracy: 100.0%
Validation accuracy: 38.4%
Minibatch loss at step 1500: 70.589180
Minibatch accuracy: 100.0%
Validation accuracy: 38.4%
Minibatch loss at step 2000: 42.809086
Minibatch accuracy: 100.0%
Validation accuracy: 38.5%
Minibatch loss at step 2500: 25.961781
Minibatch accuracy: 100.0%
Validation accuracy: 38.8%
Minibatch loss at step 3000: 15.744670
Minibatch accuracy: 100.0%
Validation accuracy: 39.0%
Minibatch loss at step 3500: 9.548479
Minibatch accuracy: 100.0%
Validation accuracy: 39.5%
Minibatch loss at step 4000: 5.790795
Minibatch accuracy: 100.0%
Validation accuracy: 40.4%
Minibatch loss at step 4500: 3.512010
Minibatch accuracy: 100.0%
Validation accuracy: 41.4%
Minibatch loss at step 5000: 2.130318
Minibatch accuracy: 100.0%

We can clearly see that we get 100% accuracy on the training data by crazily overfitting it, and we fail to generalize well for our testing data

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### NN with Dropout

In [35]:
batch_size = 128
hidden_layer_nodes = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relus = tf.nn.relu(layer1)
  post_dropout = tf.nn.dropout(relus, keep_prob=0.5)
  logits = tf.matmul(post_dropout, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (
      beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_valid_dataset, weights1) + biases1
          ), weights2)
      + biases2)


  test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_test_dataset, weights1) + biases1
          ), weights2)
      + biases2)

In [36]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 906.185913
Minibatch accuracy: 5.5%
Validation accuracy: 21.7%
Minibatch loss at step 500: 195.113754
Minibatch accuracy: 84.4%
Validation accuracy: 81.8%
Minibatch loss at step 1000: 119.433556
Minibatch accuracy: 80.5%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 70.160820
Minibatch accuracy: 82.0%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 42.529278
Minibatch accuracy: 86.7%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 25.720526
Minibatch accuracy: 91.4%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 15.649412
Minibatch accuracy: 94.5%
Validation accuracy: 85.7%
Minibatch loss at step 3500: 9.790589
Minibatch accuracy: 89.8%
Validation accuracy: 86.1%
Minibatch loss at step 4000: 6.115727
Minibatch accuracy: 91.4%
Validation accuracy: 86.6%
Minibatch loss at step 4500: 3.929503
Minibatch accuracy: 92.2%
Validation accuracy: 86.8%
Minibatch loss at step 5000: 2.625267
Minibatch accuracy: 91.4%
Validatio

### Overfit case with dropout

In [5]:
batch_size = 32
hidden_layer_nodes = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relus = tf.nn.relu(layer1)
  post_dropout = tf.nn.dropout(relus, keep_prob=0.5)
  logits = tf.matmul(post_dropout, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (
      beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_valid_dataset, weights1) + biases1
          ), weights2)
      + biases2)


  test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(
              tf.matmul(tf_test_dataset, weights1) + biases1
          ), weights2)
      + biases2)

In [40]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # BAD BAD BAD, just doing this mod to see how dropout helps overfitting
    offset = offset % 5
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 753.943970
Minibatch accuracy: 15.6%
Validation accuracy: 25.2%
Minibatch loss at step 500: 194.654709
Minibatch accuracy: 100.0%
Validation accuracy: 50.3%
Minibatch loss at step 1000: 118.183601
Minibatch accuracy: 100.0%
Validation accuracy: 49.0%
Minibatch loss at step 1500: 71.673019
Minibatch accuracy: 100.0%
Validation accuracy: 49.0%
Minibatch loss at step 2000: 43.466507
Minibatch accuracy: 100.0%
Validation accuracy: 49.1%
Minibatch loss at step 2500: 26.373154
Minibatch accuracy: 100.0%
Validation accuracy: 48.3%
Minibatch loss at step 3000: 15.999766
Minibatch accuracy: 100.0%
Validation accuracy: 51.9%
Minibatch loss at step 3500: 9.704283
Minibatch accuracy: 100.0%
Validation accuracy: 50.3%
Minibatch loss at step 4000: 5.885230
Minibatch accuracy: 100.0%
Validation accuracy: 50.3%
Minibatch loss at step 4500: 3.569132
Minibatch accuracy: 100.0%
Validation accuracy: 50.2%
Minibatch loss at step 5000: 2.164565
Minibatch accuracy: 100.0

With the dropout, we expect that we will not overfit as much to the training data. By dropping out half of the weights, we don't allow the network to simply memorize the training dataset, forcing it to generalize at least a little bit better.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [11]:
batch_size = 128
hidden_layer_nodes = 1024
beta = 0.0000001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_nodes], stddev=sqrt(2.0/(image_size * image_size))))
  biases1 = tf.Variable(tf.zeros([hidden_layer_nodes]))
  
  weights2 = tf.Variable(tf.truncated_normal([hidden_layer_nodes, hidden_layer_nodes], stddev=sqrt(2.0/hidden_layer_nodes)))
  biases2 = tf.Variable(tf.zeros([hidden_layer_nodes]))

  weights3 = tf.Variable(
    tf.truncated_normal([hidden_layer_nodes, num_labels], stddev=sqrt(2.0/hidden_layer_nodes)))
  biases3 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  layer1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relus1 = tf.nn.relu(layer1)
  post_dropout1 = tf.nn.dropout(relus1, keep_prob=0.5)
  layer2 = tf.matmul(post_dropout1, weights2) + biases2
  relus2 = tf.nn.relu(layer2)
  post_dropout2 = tf.nn.dropout(relus2, keep_prob=0.5)
  logits = tf.matmul(post_dropout2, weights3) + biases3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.04, global_step, 1000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(tf.matmul(
              tf.nn.relu(
                  tf.matmul(tf_valid_dataset, weights1) + biases1), weights2)
              + biases2)
          , weights3) + biases3)

  test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(tf.matmul(
              tf.nn.relu(
                  tf.matmul(tf_test_dataset, weights1) + biases1), weights2)
              + biases2)
          , weights3) + biases3)

In [12]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.637584
Minibatch accuracy: 8.6%
Validation accuracy: 16.4%
Minibatch loss at step 1000: 0.830163
Minibatch accuracy: 75.0%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.415825
Minibatch accuracy: 89.1%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 0.496942
Minibatch accuracy: 86.7%
Validation accuracy: 85.0%
Minibatch loss at step 4000: 0.512879
Minibatch accuracy: 83.6%
Validation accuracy: 85.5%
Minibatch loss at step 5000: 0.584164
Minibatch accuracy: 82.0%
Validation accuracy: 85.9%
Minibatch loss at step 6000: 0.454035
Minibatch accuracy: 88.3%
Validation accuracy: 86.3%
Minibatch loss at step 7000: 0.500968
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 8000: 0.467915
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 9000: 0.370032
Minibatch accuracy: 89.1%
Validation accuracy: 87.0%
Minibatch loss at step 10000: 0.302937
Minibatch accuracy: 89.8%
Validation accura